In [3]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeRegressor

In [4]:
df = pd.read_csv("card_cust.csv")
df.head(2)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001,40.900749,0.818182,95.4,0.0,95.4,0.000000,0.166667,0.0,0.083333,0.00,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,10002,3202.467416,0.909091,0.0,0.0,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0


### 전처리

In [ ]:
df.isna().sum()

In [6]:
df["MINIMUM_PAYMENTS"] = df["MINIMUM_PAYMENTS"].fillna(df["MINIMUM_PAYMENTS"].mean())

In [7]:
df_base = df.copy()

### Q1.

In [ ]:
df_q1 = df[["TENURE", "BALANCE", "CREDIT_LIMIT"]]
df_q1.head(2)

In [ ]:
df_q1["TENURE"].unique()

In [27]:
df_q1_corr = df_q1.groupby("TENURE")[["BALANCE", "CREDIT_LIMIT"]].corr()
df_q1_corr

BALANCE  CREDIT_LIMIT
TENURE                                     
6.0    BALANCE       1.000000      0.868056
       CREDIT_LIMIT  0.868056      1.000000
7.0    BALANCE       1.000000      0.948405
       CREDIT_LIMIT  0.948405      1.000000
8.0    BALANCE       1.000000      0.820696
       CREDIT_LIMIT  0.820696      1.000000
9.0    BALANCE       1.000000      0.085474
       CREDIT_LIMIT  0.085474      1.000000
10.0   BALANCE       1.000000      0.291482
       CREDIT_LIMIT  0.291482      1.000000
11.0   BALANCE       1.000000      0.380360
       CREDIT_LIMIT  0.380360      1.000000
12.0   BALANCE       1.000000      0.460833
       CREDIT_LIMIT  0.460833      1.000000

In [18]:
df_q1_corr.loc[df_q1_corr["BALANCE"] != 1, "BALANCE"].round(2).max()

0.95

In [ ]:
df_q1_corr.iloc[1::2, ] # 처음부터 끝까지 인덱스 2씩 증가

In [21]:
df_q1_corr.iloc[::2, 1].round(2).max() # 외우지 않아도 됨

0.95

In [25]:
df_q1_corr.xs("BALANCE", level = 1)["CREDIT_LIMIT"].round(2).max() # 외우지 않아도 됨

0.95

### Q2.

In [33]:
df_q2 = df_base.drop(columns = "CUST_ID")

In [34]:
arr_q2_nor = StandardScaler().fit_transform(df_q2)

In [38]:
k = 2
model_kmeans = KMeans(n_clusters = k, random_state = 1234)
model_kmeans.fit(arr_q2_nor)
silhouette_score(arr_q2_nor, labels = model_kmeans.labels_)

0.21544722357637516

In [ ]:
ls_k = [2, 3, 4, 5]

ls_sil = []
for k in ls_k:
    model_kmeans = KMeans(n_clusters = k, random_state = 1234)
    model_kmeans.fit(arr_q2_nor)
    val_sil = silhouette_score(arr_q2_nor, labels = model_kmeans.labels_)    
    ls_sil = ls_sil + [val_sil]

In [ ]:
ser_sil = pd.Series(ls_sil, index = ls_k)
best_k = ser_sil.idxmax()
best_k

In [ ]:
model_kmeans_b = KMeans(n_clusters = best_k, random_state = 1234)
model_kmeans_b.fit(arr_q2_nor)

In [49]:
df_q2["cluster"] = model_kmeans_b.labels_

In [51]:
df_q2.groupby("cluster")["ONEOFF_PURCHASES"].mean().round(2)

cluster
0     272.26
1    2156.47
Name: ONEOFF_PURCHASES, dtype: float64

### Q3.

In [52]:
df_train = df_base.loc[(df_base["CUST_ID"] % 4) != 0, ]
df_test  = df_base.loc[(df_base["CUST_ID"] % 4) == 0, ]
len(df_train), len(df_test)

(752, 248)

In [54]:
model_dt = DecisionTreeRegressor(random_state = 1234)
model_dt.fit(X = df_train.drop(columns = ["CUST_ID", "ONEOFF_PURCHASES"]),
             y = df_train["ONEOFF_PURCHASES"])
pred = model_dt.predict(df_test.drop(columns = ["CUST_ID", "ONEOFF_PURCHASES"]))

In [56]:
y_t = df_test["ONEOFF_PURCHASES"]
y_p = pred

In [59]:
from sklearn.metrics import mean_squared_error
round(mean_squared_error(y_true = y_t, y_pred = y_p) ** 0.5, 1)

1039.2

In [62]:
# y_t - y_p # E, Error
# (y_t - y_p) ** 2 # SE, Squared Error
# ((y_t - y_p) ** 2).mean() # MSE, Mean Squared Error
((y_t - y_p) ** 2).mean() ** 0.5 # RMSE, Root Mean Squared Error

1039.193967231063